In [54]:
import pandas as pd 
pd.set_option('display.max_columns', None)


 Usaremos la base de datos de los  [Cómputos Distritales para la Elección Presidencial de 2018 a nivel casilla](https://computos2018.ine.mx/#/descargaBase) del Instituto Nacional Electoral (INE) de México para analizar los votos obtenidos por los candidatos presidenciales en las elecciones de 2018.

In [55]:
Votos = pd.read_csv("presidencia.csv", sep="|", 
                    encoding="latin1",engine="python",skiprows=6)
print(Votos["NOMBRE_ESTADO"].unique())
Votos.head()

['AGUASCALIENTES' 'BAJA CALIFORNIA' 'BAJA CALIFORNIA SUR' 'CAMPECHE'
 'COAHUILA' 'COLIMA' 'CHIAPAS' 'CHIHUAHUA' 'CIUDAD DE MÉXICO' 'DURANGO'
 'GUANAJUATO' 'GUERRERO' 'HIDALGO' 'JALISCO' 'MÉXICO' 'MICHOACÁN'
 'MORELOS' 'NAYARIT' 'NUEVO LEÓN' 'OAXACA' 'PUEBLA' 'QUERÉTARO'
 'QUINTANA ROO' 'SAN LUIS POTOSÍ' 'SINALOA' 'SONORA' 'TABASCO'
 'TAMAULIPAS' 'TLAXCALA' 'VERACRUZ' 'YUCATÁN' 'ZACATECAS']


,CLAVE_CASILLA,CLAVE_ACTA,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO,NOMBRE_DISTRITO,SECCION,ID_CASILLA,TIPO_CASILLA,EXT_CONTIGUA,CASILLA,NUM_ACTA_IMPRESO,PAN,PRI,PRD,PVEM,PT,MOVIMIENTO CIUDADANO,NUEVA ALIANZA,MORENA,ENCUENTRO SOCIAL,PAN_PRD_MC,PAN_PRD,PAN_MC,PRD_MC,PRI_PVEM_NA,PRI_PVEM,PRI_NA,PVEM_NA,PT_MORENA_PES,PT_MORENA,PT_PES,MORENA_PES,CAND_IND_01,CAND_IND_02,CNR,VN,TOTAL_VOTOS_CALCULADOS,LISTA_NOMINAL_CASILLA,OBSERVACIONES,MECANISMOS_TRASLADO,FECHA_HORA,Unnamed: 42,Unnamed: 43
0,'010000M0100',010000M010001,1,AGUASCALIENTES,0,,0,1,M,0,,1,247,37,8,1,24,4,0,378,5,27,1,2,0,6,0,1,0,47,7,1,1,0,26,5,9,837,1683,-,-,05/07/18 14:00:00.000000,NaN,NaN
1,'010338B0100',010338B010002,1,AGUASCALIENTES,1,JESÚS MARÍA,338,1,B,0,Rural,2,62,43,2,1,21,4,3,153,5,1,0,0,1,0,0,0,0,5,1,0,1,0,14,0,20,337,587,Cotejo (Levantada en Casilla),F,04/07/18 09:23:10.081000,NaN,NaN
2,'010338C0100',010338C010002,1,AGUASCALIENTES,1,JESÚS MARÍA,338,1,C,0,Rural,2,81,36,1,3,11,3,3,159,9,1,0,0,0,1,0,0,0,1,2,0,0,0,18,0,6,335,586,Cotejo (Levantada en Casilla),F,04/07/18 09:26:33.092000,NaN,NaN
3,'010338C0200',010338C020002,1,AGUASCALIENTES,1,JESÚS MARÍA,338,2,C,0,Rural,2,73,28,2,3,19,4,5,136,4,2,1,0,0,0,0,1,0,5,0,0,2,0,13,0,7,305,586,Cotejo (Levantada en Casilla),F,04/07/18 09:31:31.102000,NaN,NaN
4,'010339B0100',010339B010002,1,AGUASCALIENTES,1,JESÚS MARÍA,339,1,B,0,Rural,2,75,43,2,5,3,11,1,151,8,3,1,1,0,0,1,0,0,0,3,0,0,0,14,0,6,328,552,Cotejo (Levantada en Casilla),F-D,04/07/18 09:34:53.187000,NaN,NaN


La base Votos no tiene la columna municipios, por lo que nos ayudaremos de la base de LISTADO_CASILLAS_2018.csv para agregar esa información. a la base Votos.

In [56]:
#BASE DE DATOS CON LOS NOMBRES DE LOS ESTADOS SIN CARACTERES EXTRAÑOS
estados = pd. read_csv("EDos.csv", sep=",")


Listado_casillas = pd.read_csv("LISTADO_CASILLAS_2018.csv", sep=",", 
                              encoding="latin1",engine="python")

print(estados.head())
print(Listado_casillas.head())

   ID_ESTADO        NOMBRE_ESTADO
0          1       AGUASCALIENTES
1          2      BAJA CALIFORNIA
2          3  BAJA CALIFORNIA SUR
3          4             CAMPECHE
4          5             COAHUILA
               ID  ID_ESTADO  ID_DISTRITO_FEDERAL  SECCION TIPO_CASILLA  \
0  A01010338B0100          1                    1      338            B   
1  A01010338C0100          1                    1      338            C   
2  A01010338C0200          1                    1      338            C   
3  A01010339B0100          1                    1      339            B   
4  A01010339C0100          1                    1      339            C   

   ID_CASILLA  EXT_CONTIGUA  TIPO_SECCION  LISTA_NOMINAL  ID_MUNICIPIO  \
0           1             0           2.0            587             2   
1           1             0           2.0            586             2   
2           2             0           2.0            586             2   
3           1             0           2.0        

In [64]:
Listado_casillas_2= pd.merge(Listado_casillas, estados, on = "ID_ESTADO", how = "left")

# crear funcion que creee un Id unico por casilla para unir la base de datos 
def crear_id_casilla(df):
    df["ID"] = ("A" + df["ID_ESTADO"].astype(str).str.zfill(2) +
                              df["SECCION"].astype(str).str.zfill(4) +
                              df["TIPO_CASILLA"] + 
                              df["ID_CASILLA"].astype(str).str.zfill(2))
    return df

Listado_casillas_2 = crear_id_casilla(Listado_casillas_2)
Votos = crear_id_casilla(Votos)

Votos.columns


Index(['CLAVE_CASILLA', 'CLAVE_ACTA', 'ID_ESTADO', 'NOMBRE_ESTADO',
       'ID_DISTRITO', 'NOMBRE_DISTRITO', 'SECCION', 'ID_CASILLA',
       'TIPO_CASILLA', 'EXT_CONTIGUA', 'CASILLA', 'NUM_ACTA_IMPRESO', 'PAN',
       'PRI', 'PRD', 'PVEM', 'PT', 'MOVIMIENTO CIUDADANO', 'NUEVA ALIANZA',
       'MORENA', 'ENCUENTRO SOCIAL', 'PAN_PRD_MC', 'PAN_PRD', 'PAN_MC',
       'PRD_MC', 'PRI_PVEM_NA', 'PRI_PVEM', 'PRI_NA', 'PVEM_NA',
       'PT_MORENA_PES', 'PT_MORENA', 'PT_PES', 'MORENA_PES', 'CAND_IND_01',
       'CAND_IND_02', 'CNR', 'VN', 'TOTAL_VOTOS_CALCULADOS',
       'LISTA_NOMINAL_CASILLA', 'OBSERVACIONES', 'MECANISMOS_TRASLADO',
       'FECHA_HORA', 'Unnamed: 42', 'Unnamed: 43', 'ID'],
      dtype='object')

In [115]:
# ahora voy a hacer una union de las dos bases de datos

Votos_completo = pd.merge(Votos, Listado_casillas_2[["ID", "NOMBRE_ESTADO", "ID_MUNICIPIO"]], 
                          on = "ID", how = "left")

filas_na = Votos_completo["NOMBRE_ESTADO_y"].isna().sum()
print("Renglones con NA:", filas_na )
print("Porcentaje renglones con NA:", filas_na / Votos_completo.shape[0] * 100)
Votos_completo.columns

Renglones con NA: 37
Porcentaje renglones con NA: 0.02129337085570576


Index(['CLAVE_CASILLA', 'CLAVE_ACTA', 'ID_ESTADO', 'NOMBRE_ESTADO_x',
       'ID_DISTRITO', 'NOMBRE_DISTRITO', 'SECCION', 'ID_CASILLA',
       'TIPO_CASILLA', 'EXT_CONTIGUA', 'CASILLA', 'NUM_ACTA_IMPRESO', 'PAN',
       'PRI', 'PRD', 'PVEM', 'PT', 'MOVIMIENTO CIUDADANO', 'NUEVA ALIANZA',
       'MORENA', 'ENCUENTRO SOCIAL', 'PAN_PRD_MC', 'PAN_PRD', 'PAN_MC',
       'PRD_MC', 'PRI_PVEM_NA', 'PRI_PVEM', 'PRI_NA', 'PVEM_NA',
       'PT_MORENA_PES', 'PT_MORENA', 'PT_PES', 'MORENA_PES', 'CAND_IND_01',
       'CAND_IND_02', 'CNR', 'VN', 'TOTAL_VOTOS_CALCULADOS',
       'LISTA_NOMINAL_CASILLA', 'OBSERVACIONES', 'MECANISMOS_TRASLADO',
       'FECHA_HORA', 'Unnamed: 42', 'Unnamed: 43', 'ID', 'NOMBRE_ESTADO_y',
       'ID_MUNICIPIO'],
      dtype='object')

No se realiza bien la unión entre las dos bases de datos, pero el porcentaje de  renglones con NA es minimo. Por tanto omitiremos estos renglones.

In [ ]:
# Vamos a filtrar el conteo en casa casila junto con algunos datos de identificacion  y quitar los renglones 
# que contienen NA
Votos_completo = Votos_completo.dropna(subset=["NOMBRE_ESTADO_y"])
Votos_completo = Votos_completo.loc[:,
[
   "ID",                     "ID_ESTADO",              "NOMBRE_ESTADO_y",        "ID_MUNICIPIO",          
   "PAN",                    "PRI",                    "PRD",                    "PVEM",                  
   "PT",                     "MOVIMIENTO CIUDADANO",   "NUEVA ALIANZA",          "MORENA",                
   "ENCUENTRO SOCIAL",       "PAN_PRD_MC",             "PAN_PRD",                "PAN_MC",                
   "PRD_MC",                 "PRI_PVEM_NA",            "PRI_PVEM",               "PRI_NA",                
   "PVEM_NA",                "PT_MORENA_PES",          "PT_MORENA",              "PT_PES",                
   "MORENA_PES",             "CAND_IND_01",            "CAND_IND_02",            "CNR",                   
   "VN",                     "TOTAL_VOTOS_CALCULADOS", "LISTA_NOMINAL_CASILLA" ]]

Ahora lo que vamos a hacer es obtener el total de votos por municipio para cada candidato presidencial. Los votos se suman de la siguiente manera:

| Candidato | Coaliciones a Sumar |    
|:---|---:|
|Andres Manuel Lopez Obrador|PT, MORENA, ENCUENTRO SOCIAL, PT-MORENA-PES, PT-MORENA, PT-PES,MORENA-PES|  
|Ricardo Anaya Cortés|PAN, PRI, MOVIMIENTO CIUDADANO, PAN-PRD-MC, PAN-PRD, PAN-MC, PRD-MC|
 
```R